In [1]:
%pip install opencv-python
%pip install mediapipe



Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp

In [3]:
## initialize pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [10]:
import cv2
import numpy as np
import tensorflow as tf
import time

def process_video(video_path, model, input_size):
    cap = cv2.VideoCapture(video_path)
    landmarks_list = []
    frame_num = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        input_image = cv2.resize(frame, (input_size, input_size))
        input_image = np.expand_dims(input_image, axis=0)
        input_image = tf.cast(input_image, dtype=tf.int32)  # or tf.float32 if using a model expecting float32

        start_time = time.time()
        keypoints_with_scores = model(input_image)
        end_time = time.time()

        # Calculate inference time
        inference_time = end_time - start_time

        # Extract keypoints
        keypoints = keypoints_with_scores['output_0'].numpy()[0, 0, :, :]
        landmarks_list.append({'frame': frame_num, 'keypoints': keypoints, 'inference_time': inference_time})

        frame_num += 1

        # Display the frame with keypoints (optional)
        for keypoint in keypoints:
            x, y, _ = keypoint
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
        cv2.imshow('Pose Estimation', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return landmarks_list


In [11]:
def get_landmark_by_index(landmarks, index):
    """Extract a specific landmark by its index."""
    return [landmark['keypoints'][index] for landmark in landmarks]


In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from math import atan2, degrees

# Define function to calculate angle between three points
def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    ang = atan2(c[1] - b[1], c[0] - b[0]) - atan2(a[1] - b[1], a[0] - b[0])
    ang = abs(degrees(ang))
    return ang if ang <= 180 else 360 - ang

# Initialize pose model (replace with your model initialization)
# For example: pose = mp_pose.Pose()

# Path to your video
video_path = 'path_to_your_video.mp4'

# Open video file
cap = cv2.VideoCapture(video_path)
landmarks_list = []
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    try:
        # Process frame for pose detection
        pose_results = pose.process(frame_rgb)
        
        # Extract landmarks
        landmarks = pose_results.pose_landmarks.landmark if pose_results.pose_landmarks else []
        keypoints = [(landmark.x, landmark.y) for landmark in landmarks]
        
        # Append landmarks and frame number to the list
        landmarks_list.append({'frame': frame_num, 'keypoints': keypoints})
        
        # Draw landmarks on frame
        # (Replace this with your own drawing code if needed)
        # mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Display the frame
        cv2.imshow('Output', frame)
        
    except Exception as e:
        print(f"Error processing frame {frame_num}: {e}")
    
    frame_num += 1

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save landmarks to CSV
landmarks_df = pd.DataFrame(landmarks_list)
landmarks_df.to_csv('landmarks_output.csv', index=False)

# Load landmarks from CSV
df = pd.read_csv('landmarks_output.csv')

# Calculate and plot angles
# Example: Calculate angle between landmarks 11 (shoulder), 12 (elbow), 13 (wrist) for upper arm
def plot_angles(df, landmark_indices, angle_name):
    angles = []
    for _, row in df.iterrows():
        keypoints = row['keypoints']
        if len(keypoints) >= max(landmark_indices):
            p1, p2, p3 = [keypoints[i] for i in landmark_indices]
            angle = calculate_angle(p1, p2, p3)
            angles.append(angle)
        else:
            angles.append(None)
    
    plt.figure(figsize=(10, 6))
    plt.plot(df['frame'], angles, label=angle_name)
    plt.xlabel('Frame Number')
    plt.ylabel('Angle (degrees)')
    plt.title(f'Angle of {angle_name} Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example indices for landmarks (adjust as needed)
landmark_indices_upper_body = [11, 12, 13]  # Example for upper arm angle
plot_angles(df, landmark_indices_upper_body, 'Upper Arm Angle')


EmptyDataError: No columns to parse from file